In [ ]:
import os
import sys
# os.chdir('/Users/ludo/Desktop/animalai/animalai')
os.chdir('/Users/ludo/Desktop/animalai/animalai/neuro')

In [ ]:


sys.path.insert(0, "/Users/ludo/Desktop/animalai/animalai/animalai_train")
sys.path.insert(1, "/Users/ludo/Desktop/animalai/animalai/animalai")
from animalai.envs.gym.environment import AnimalAIGym
import random
from animalai.envs.arena_config import ArenaConfig
from centroidtracker import CentroidTracker

import yaml
import copy
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


# CONFIG
env_path = '../env/aaiagain'
worker_id = 2
#random.randint(1, 100)
# competition_envs = os.listdir('../competition_configurations/')

seed = 1


# yaml_path = '../examples/rand_env.yml'
# ac = ArenaConfig(yaml_path)
yaml_path = "../competition_configurations/5-11-3.yml" #'configurations/curriculum/0.yaml'
# ac = ArenaConfig(arena_path)
# ac.arenas[1] = copy.deepcopy(ac.arenas[0])
# yaml_path = 'configurations/curriculum4/19.yml'
# x = ['1-11-1.yml']
# yaml_path = "../competition_configurations/1-11-1.yml" #'configurations/curriculum/0.yaml'

ac = ArenaConfig(yaml_path)
env = AnimalAIGym(environment_filename=env_path,
              worker_id=worker_id,
              n_arenas=1,
              arenas_configurations=ac,
                use_visual=False,
                seed=seed,
                 grayscale=False,
                 resolution=1024)

x = env.step([[0,0]])

In [ ]:
from animalai.envs.cvis_img import ExtractFeatures
%matplotlib inline
import cv2
import time
from IPython import display
for i in range(10):
    x = env.step([[0,2]])
plt.imshow(env.visual_obs)


In [ ]:
from animalai.envs.cvis_test import ExtractFeatures

ef = ExtractFeatures(display=True, training=False)
# res = ef.run_dual(env.visual_obs)
res = ef.run_test(env.visual_obs)
# plt.imshow(res[0])
# plt.imshow(env.visual_obs)
# class HSV:
# 	"""Color lower and upper bounds in HSV format"""

# 	@classmethod
# 	def __getattribute__(cls,attr):
# 		return [np.array(i) for i in getattr(cls, attr)]

# 	green = [[33,80,40], [102,255,255]]
# 	red = [[0,162,142], [179,203,188]]
# 	grey = [[0,0,0], [0,0,224]]
# 	brown = [[7,53,40], [18,87,121]]
# hsv_cls = HSV()

# img = env.visual_obs
# img = (img*255)[:,:,::-1].astype(np.uint8)
# hsv = hsv_cls.red
# hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# mask = cv2.inRange(hsv_img, hsv[0], hsv[1])
# res = cv2.bitwise_and(hsv_img, hsv_img, mask=mask)
# plt.imshow(res[:,:,2])
# plt.imshow(hsv_img)
# im = np.ascontiguousarray(
#     cv2.imdecode(np.frombuffer(
#         res[0].tobytes(), np.float32), -1))
# im
# plt.imshow(res[0])

In [ ]:
im = res[0].reshape(84,84).astype(np.float32)
bf = np.frombuffer(im.tobytes(order="C"), np.float32).reshape(84,84)
plt.imshow(bf)

In [ ]:
# res[res > 0] = 1
# plt.imshow(res[:,:,2])
# res[:,:,2][45]
from PIL import Image
import io
# image_bytearray = bytearray(bf)
# image = Image.open(io.BytesIO(image_bytearray))
# image.load()
byteImgIO = io.BytesIO()
byteImg = Image.fromarray(im.astype(np.uint8))
byteImg.save(byteImgIO, "PNG")
byteImgIO.seek(0)
byteImg = byteImgIO.read()
image_bytearray = bytearray(byteImg)
image = Image.open(io.BytesIO(image_bytearray))
# Normally Image loads lazily, this forces it to do loading in the timer scope.
image.load()
s = np.array(image, dtype=np.float32).reshape(84,84, 1)
s = np.mean(s, axis=2)
s = np.reshape(s, [s.shape[0], s.shape[1]])
plt.imshow(s)

In [ ]:
ct = CentroidTracker(75)
_ = env.reset()

In [ ]:
%matplotlib inline
from animalai.envs.cvis_test import ExtractFeatures
import cv2
import time
from IPython import display

ef = ExtractFeatures(display=True, training=False)
env.step([[0,0]])
res = ef.run(env.visual_obs)
plt.imshow(ef.img[:,:,::-1])
# for i in range(10):

#     env.step([[1,0]])
#     res = ef.run(env.visual_obs, i)['wall']
# #     print(res)
#     convert = lambda x: [x[0]*84, x[1]*84, (x[0]+x[2])*84, (x[1]+x[3])*84]
#     res  = [(convert(i[0])) for i in res]
#     ids = ct.update(res)
#     img = ef.img.copy()
#     if res:
#         for c,i in ids.items():
#             cv2.putText(img, str(c), (i[0], i[1]),
#                         cv2.FONT_HERSHEY_SIMPLEX,
#                         0.5,
#                         (255,255,255),
#                         1)
#     plt.imshow(img[:,:,::-1])
#     display.display(plt.gcf())

#     display.clear_output(wait=True)
#     time.sleep(0.15)


In [ ]:
saved_img = img[:,:,::-1]
plt.imshow(saved_img)
plt.savefig('/Users/ludo/Desktop/savedimg.png',bbox_inches='tight',transparent=True, pad_inches=0)

In [ ]:
import cv2
img = (env.visual_obs*255)[:,:,::-1].astype(np.uint8)
class HSV:
	"""Color lower and upper bounds in HSV format"""
	@classmethod
	def __getattribute__(cls,attr):
		return [np.array(i) for i in getattr(cls, attr)]

	green = [[33,80,40], [102,255,255]]
	red = [[125,16,88], [179,255,255]]
	grey = [[0,0,0], [0,0,224]]
	brown = [[7,53,40], [18,87,121]]

hsv_cls = HSV()
hsv = hsv_cls.grey
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_img, hsv[0], hsv[1])
res = cv2.bitwise_and(hsv_img, hsv_img, mask=mask)[:,:,2]
ctr,hier = cv2.findContours(res,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
for c in ctr:
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),1)
plt.imshow(img[:,:,::-1])


In [ ]:
from animalai.envs.cvis import ExtractFeatures
ef = ExtractFeatures(display=True, training=False)
ef.run(env.visual_obs)

In [ ]:
x[3]["batched_step_result"].obs[1]

In [ ]:
try:
    next(i for i in [1,2,3] if i==4)
except StopIteration:
    print('ya')

In [ ]:
img = (env.visual_obs*255)[:,:,::-1].astype(np.uint8)

plt.imshow(img[:,:,::-1])


In [ ]:
convert = lambda x: [x[0], x[1], x[0]+x[2], x[1]+x[3]]

convert(ef.run(env.visual_obs)[0][0])


In [ ]:
ef.run(img)[0][0]

In [ ]:
plt.imshow(ef.img)

In [ ]:
class State:
    obj = [[4,3,2], [3,4,1]]
    velocity = [1,2,3]
    reward = 0
    done = False
    def __repr__(self):
        return f"""
        Velocity: {self.velocity}
        Reward: {self.reward}
        Done: {self.done}
        """
    def __getattribute__(self, item):
        if item=='boxes':
            return [
                i[0] for i in super(
                    State, self).__getattribute__('obj')]
        elif item=='obj_types':
            return set([
                i[1] for i in super(
                    State, self).__getattribute__('obj')])
        elif item=='ids':
            return [
                i[2] for i in super(
                    State, self).__getattribute__('obj')]
          
            
        return super(State, self).__getattribute__(item)
    def __setattr__(self, item, value):
        return super(State, self).__setattr__('item', value)
        print("YAS")


In [ ]:
s = State()
s.reward = 4

In [ ]:
s.boxes

In [ ]:
ids[0][1]

In [ ]:
import cv2
img = (env.visual_obs*255)[:,:,::-1].astype(np.uint8)

cv2.imwrite("/Users/ludo/Desktop/img3.png",img)

In [ ]:
boxes = """[[0.0, 0.5357142857142857, 0.19047619047619047, 0.023809523809523808], [0.6309523809523809, 0.4642857142857143, 0.14285714285714285, 0.07142857142857142], [0.2976190476190476, 0.0, 0.21428571428571427, 0.6071428571428571]]
[[0.0, 0.5357142857142857, 0.08333333333333333, 0.011904761904761904], [0.5357142857142857, 0.47619047619047616, 0.14285714285714285, 0.047619047619047616], [0.21428571428571427, 0.0, 0.20238095238095238, 0.6071428571428571]]
[[0.4523809523809524, 0.47619047619047616, 0.13095238095238096, 0.047619047619047616], [0.11904761904761904, 0.0, 0.21428571428571427, 0.6071428571428571]]
[[0.35714285714285715, 0.47619047619047616, 0.13095238095238096, 0.047619047619047616], [0.011904761904761904, 0.0, 0.2261904761904762, 0.6071428571428571]]
[[0.2619047619047619, 0.47619047619047616, 0.13095238095238096, 0.047619047619047616], [0.0, 0.0, 0.13095238095238096, 0.6071428571428571]]
[[0.35714285714285715, 0.47619047619047616, 0.13095238095238096, 0.047619047619047616], [0.0, 0.0, 0.2261904761904762, 0.6190476190476191]]
[[0.4523809523809524, 0.47619047619047616, 0.13095238095238096, 0.047619047619047616], [0.08333333333333333, 0.0, 0.2261904761904762, 0.6190476190476191]]
[[0.0, 0.5357142857142857, 0.07142857142857142, 0.011904761904761904], [0.5357142857142857, 0.4642857142857143, 0.13095238095238096, 0.07142857142857142], [0.15476190476190477, 0.0, 0.2261904761904762, 0.6190476190476191]]
[[0.44047619047619047, 0.4642857142857143, 0.14285714285714285, 0.07142857142857142], [0.0, 0.0, 0.27380952380952384, 0.6309523809523809]]
[[0.0, 0.5357142857142857, 0.047619047619047616, 0.011904761904761904], [0.5357142857142857, 0.4642857142857143, 0.14285714285714285, 0.07142857142857142], [0.05952380952380952, 0.0, 0.2857142857142857, 0.6428571428571429]]
[[0.44047619047619047, 0.4642857142857143, 0.14285714285714285, 0.07142857142857142], [0.0, 0.0, 0.21428571428571427, 0.6666666666666666]]
[[0.5357142857142857, 0.4642857142857143, 0.14285714285714285, 0.07142857142857142], [0.0, 0.0, 0.30952380952380953, 0.6904761904761905]]
[[0.6309523809523809, 0.4642857142857143, 0.14285714285714285, 0.07142857142857142], [0.0, 0.0, 0.40476190476190477, 0.7142857142857143]]
[[0.7142857142857143, 0.4642857142857143, 0.16666666666666666, 0.07142857142857142], [0.0, 0.0, 0.4880952380952381, 0.7619047619047619]]
[[0.8214285714285714, 0.4642857142857143, 0.17857142857142858, 0.07142857142857142], [0.0, 0.0, 0.5833333333333334, 0.8690476190476191]]"""

In [ ]:
convert = lambda x: [x[0]*84, x[1]*84, (x[0]+x[2])*84, (x[1]+x[3])*84]
lb = [[convert(j) for j in eval(i)] for i in boxes.split("\n")]
ct = CentroidTracker(75)


In [ ]:

for i in lb:
    print(ct.update(i))

In [ ]:
import os
x= [i for i in os.listdir('../competition_configurations/') if i[0]=='5']
x = sorted(x, key=lambda i: int(i.split('-')[1]))
x = [ArenaConfig(f"../competition_configurations/{i}") for i in x]
# ac = ArenaConfig(f"../competition_configurations/{x[0]}")
# for c, i in enumerate(x[1:]):
#     ac.arenas[c+1] = ArenaConfig(f"../competition_configurations/{i}").arenas[0]

for i in x:
    for _ in range(10):
        env.step([[0,1]])
    env.reset(i)